In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import pandas as pd

In [15]:
train_path = '/kaggle/input/adobetraindata/behaviour_simulation_train.csv'
test_path = '/kaggle/input/inter-iit-mid-prep-adobe/problem_1_test_dataset/behaviour_simulation_test_company.xlsx'

In [16]:
test_dataset = pd.read_excel(test_path)

In [17]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                10000 non-null  int64 
 1   date              10000 non-null  object
 2   content           10000 non-null  object
 3   username          10000 non-null  object
 4   media             10000 non-null  object
 5   inferred company  10000 non-null  object
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


In [51]:
train_dataset = pd.read_csv(train_path)

In [52]:
train_dataset

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
...,...,...,...,...,...,...,...
299995,299996,2019-09-07 16:18:10,0,Barcelona Star Expected To Return Against Vale...,IndependentNGR,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
299996,299997,2018-02-23 11:24:36,46,Kjeld Nuis of #NED is golden again... This tim...,CBCOlympics,[Video(thumbnailUrl='https://pbs.twimg.com/med...,cbc
299997,299998,2020-11-11 20:18:15,261,Grateful 🙌🏾 to have <mention> officially join ...,TeamCoachBuzz,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
299998,299999,2019-10-29 10:44:00,119,The US Air Force's mysterious space plane just...,Independent,[Video(thumbnailUrl='https://pbs.twimg.com/med...,independent


In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb
# Load data
train_data = train_dataset.copy()
test_data = test_dataset.copy()
# Bin the likes into categories
# Define the cap threshold
cap_value = 1000000
# Apply the cap to the 'likes' column
train_data['likes_capped'] = train_data['likes'].apply(lambda x: min(x, cap_value))
# Check the distribution of the capped 'likes' column
print(train_data['likes_capped'].describe())
bins = [0, 100, 1000, 10000, cap_value]
labels = ['0-100', '101-1k','1k-10k','10k+']
train_data['likes_binned'] = pd.cut(train_data['likes_capped'], bins=bins, labels=labels, include_lowest=True)
# Sample 10% of the data, stratified by binned likes
sample_fraction = 0.01  # Change this to 1.0 for 100%
train_data_sampled, _ = train_test_split(train_data, test_size=1-sample_fraction, random_state=42, stratify=train_data['likes_binned'])
train_data_sampled = train_data_sampled.reset_index(drop=1)
# train_data_sampled = train_data.copy()

# Check distribution
print(train_data_sampled['likes_binned'].value_counts(normalize=True))


count    300000.000000
mean        773.364793
std        4931.463419
min           0.000000
25%           3.000000
50%          76.000000
75%         364.000000
max      560193.000000
Name: likes_capped, dtype: float64
likes_binned
0-100     0.548667
101-1k    0.327333
1k-10k    0.111667
10k+      0.012333
Name: proportion, dtype: float64


In [54]:
import re
def remove_company_name(username, companies=train_data_sampled['inferred company']):
    """
    Remove company name from a username.
    
    Args:
    username (str): The username to process.
    companies (list): A list of company names to look for.
    
    Returns:
    str: The username with the company name removed, if found.
    """
    compnaies = list(set(companies))
    # Sort companies by length (longest first) to avoid partial matches
    sorted_companies = sorted(companies, key=len, reverse=True)
    
    # Convert username to lowercase for case-insensitive matching
    lowercase_username = username.lower()
    
    for company in sorted_companies:
        # Convert company to lowercase and remove spaces
        company_pattern = company.lower().replace(' ', '')
        
        # Check if the company name is in the username
        if company_pattern in lowercase_username:
            # Remove the company name (case-insensitive)
            return re.sub(re.escape(company_pattern), '', username, flags=re.IGNORECASE)
    
    # If no company name was found, return the original username
    return username

In [55]:
train_data_sampled['username_cleaned'] = train_data_sampled['username'].apply(remove_company_name)

In [56]:
try:
    train_data_sampled['old_username'],train_data_sampled['username'] = train_data_sampled['old_username'],train_data_sampled['username_cleaned']
except:
    train_data_sampled['old_username'],train_data_sampled['username'] = train_data_sampled['username'],train_data_sampled['username_cleaned']

In [58]:
!pip install bertopic tweet-preprocessor

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00


In [24]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 40.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tsfresh 0.20.3 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [25]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import networkx as nx
from node2vec import Node2Vec
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import emoji

# Ensure you've downloaded necessary NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
# def preprocess_text(text):
#     # Convert to lowercase
#     text = text.lower()
#     # Remove URLs
#     text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
#     # Remove user mentions
#     text = re.sub(r'@\w+', '', text)
#     # Remove non-letter characters
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     # Remove extra spaces
#     text = re.sub(r'\s+', ' ', text).strip()
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     text = ' '.join([word for word in text.split() if word not in stop_words])
#     return text

# def get_roberta_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).numpy().flatten()

# def get_datetime_features(date_str):
#     date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
#     return np.array([
#         date.year, date.month, date.day, date.hour, date.minute,
#         date.weekday(), date.timetuple().tm_yday,
#         np.sin(2 * np.pi * date.hour / 24),  # Cyclical encoding for hour
#         np.cos(2 * np.pi * date.hour / 24),
#         np.sin(2 * np.pi * date.month / 12),  # Cyclical encoding for month
#         np.cos(2 * np.pi * date.month / 12)
#     ])

# def get_categorical_embedding(value, encoder):
#     return encoder.transform([[value]]).toarray().flatten()

# def create_graph_embedding(df):
#     G = nx.Graph()
#     for _, row in df.iterrows():
#         G.add_edge(row['username'], row['inferred company'])
#         for mention in re.findall(r'@(\w+)', row['content']):
#             G.add_edge(row['username'], mention)
    
#     node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
#     model = node2vec.fit(window=10, min_count=1, batch_words=4)
    
#     def get_node_embedding(node):
#         try:
#             return model.wv[node]
#         except KeyError:
#             return np.zeros(64)
    
#     df['user_graph_embedding'] = df['username'].apply(get_node_embedding)
#     df['company_graph_embedding'] = df['inferred company'].apply(get_node_embedding)
#     return df

# def extract_hashtags(text):
#     return ' '.join(re.findall(r'#(\w+)', text))

# def extract_emoji(text):
#     return ' '.join([c for c in text if c in emoji.UNICODE_EMOJI['en']])

# # Load the data
# df = pd.read_csv("tweets.csv")

# # Preprocess text
# df['processed_content'] = df['content'].apply(preprocess_text)
# df['hashtags'] = df['content'].apply(extract_hashtags)
# df['emojis'] = df['content'].apply(extract_emoji)

# # Combine all text data
# df['combined_text'] = df.apply(lambda row: f"{row['processed_content']} {row['username']} {row['inferred company']} {row['hashtags']} {row['emojis']}", axis=1)

# # Generate text embeddings
# df['combined_embedding'] = df['combined_text'].apply(get_roberta_embedding)

# # Generate datetime embeddings
# df['datetime_embedding'] = df['date'].apply(get_datetime_features)

# # Generate categorical embeddings
# company_encoder = OneHotEncoder(sparse=False)
# company_encoder.fit(df['inferred company'].values.reshape(-1, 1))
# df['company_embedding'] = df['inferred company'].apply(lambda x: get_categorical_embedding(x, company_encoder))

# # Generate graph embeddings
# df = create_graph_embedding(df)

# # Combine all embeddings
# df['rich_embedding'] = df.apply(lambda row: np.concatenate([
#     row['combined_embedding'],
#     row['datetime_embedding'],
#     row['company_embedding'],
#     row['user_graph_embedding'],
#     row['company_graph_embedding']
# ]), axis=1)

# print("Rich embeddings generated successfully!")
# print(f"Shape of rich embedding: {df['rich_embedding'].iloc[0].shape}")

# # Optional: Dimensionality reduction if needed
# # from sklearn.decomposition import PCA
# # pca = PCA(n_components=100)
# # df['reduced_embedding'] = list(pca.fit_transform(np.stack(df['rich_embedding'])))

# # Save the embeddings
# np.save('rich_tweet_embeddings.npy', np.stack(df['rich_embedding']))
# df[['id', 'content', 'username', 'inferred company']].to_csv('tweet_metadata.csv', index=False)

# print("Embeddings and metadata saved successfully!")

In [27]:
# tokenize_username('AAASouthJersey')

In [28]:
# set(df['username'])

In [29]:
set(train_data_sampled['username'])

{'',
 '04_en',
 '112',
 '365',
 '9A',
 'A4advisors',
 'ALKCM',
 'AMESA',
 'ANZ',
 'ARTravs',
 'ASPolice',
 'AU',
 'Aaryn_',
 'Abbie44',
 'Actually',
 'Ads',
 'Africa',
 'AhmadJawadBTH',
 'Airlines',
 'Airplanes',
 'Alerts',
 'AlexArthur1',
 'Allan247',
 'Amer',
 'America',
 'AmyMBE',
 'Andy_',
 'Anomia',
 'Arabia',
 'ArlanWasHere',
 'Arthurvw1986',
 'ArtisanFilms',
 'Arts',
 'Azure',
 'BGISD',
 'BWWings',
 'BWWingsOM',
 'Barrett',
 'Beacon',
 'BlackCom',
 'Boomin',
 'BornFDN',
 'BornUSA',
 'Brando',
 'Brett_',
 'Business',
 'ByTMobile',
 'CAGaming',
 'CJCJ_RL',
 'CRacing',
 'CSChallengeFW',
 'CX',
 'CaJBoyce',
 'Calgary',
 'Cam12Delport',
 'Cam_Cawthorne',
 'Canada',
 'CaseyTheVA',
 'Center',
 'Champion',
 'ChantalOnAir',
 'ChariTs',
 'CharlesPulliam',
 'CheetahsRugby',
 'ChionsCup',
 'ChristelFox4KC',
 'Cloud',
 'ClubAlliance',
 'CoachShawn',
 'Collab',
 'College',
 'ComRes',
 'Community',
 'Costellop',
 'Cricketnation',
 'Crowe',
 'Ctr',
 'DC',
 'Data',
 'DavidG',
 'DePaulU',
 'Defen

In [30]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from datetime import datetime
import networkx as nx
from node2vec import Node2Vec
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import emoji
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load specialized Twitter model
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

Using device: cuda


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [31]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user mentions
    text = re.sub(r'@\w+', '', text)
    # Remove non-letter characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

def get_roberta_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().flatten()
def get_roberta_embedding_batch(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def get_datetime_features(date_str):
    date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return np.array([
        date.year, date.month, date.day, date.hour, date.minute,
        date.weekday(), date.timetuple().tm_yday,
        np.sin(2 * np.pi * date.hour / 24),  # Cyclical encoding for hour
        np.cos(2 * np.pi * date.hour / 24),
        np.sin(2 * np.pi * date.month / 12),  # Cyclical encoding for month
        np.cos(2 * np.pi * date.month / 12)
    ])

def get_categorical_embedding(value, encoder):
    return encoder.transform([[value]]).flatten()

def create_graph_embedding(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        G.add_edge(row['username'], row['inferred company'])
        for mention in re.findall(r'@(\w+)', row['content']):
            G.add_edge(row['username'], mention)
    
    node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    
    def get_node_embedding(node):
        try:
            return model.wv[node]
        except KeyError:
            return np.zeros(64)
    
    df['user_graph_embedding'] = df['username'].apply(get_node_embedding)
    df['company_graph_embedding'] = df['inferred company'].apply(get_node_embedding)
    return df

def extract_hashtags(text):
    return ' '.join(re.findall(r'#(\w+)', text))

def extract_emoji(text):
    return ' '.join([c for c in text if emoji.is_emoji(c)])

In [32]:
# # Label encode the binned categories
# le = LabelEncoder()
# y = le.fit_transform(train_data_sampled['likes_binned'])

# # Define the groups based on the 'inferred_company'
# groups = train_data_sampled['inferred company']

# # Initialize GroupShuffleSplit with stratification
# gss = StratifiedGroupKFold(n_splits=5,shuffle=True, random_state=42)
# train_idx, val_idx = next(gss.split(train_data_sampled, train_data_sampled['likes_binned'], groups))
# # Perform the split
# # for train_idx, val_idx in gss.split(X_combined, y, groups):
# X_train, X_val = train_data_sampled.iloc[train_idx].reset_index(drop=True), train_data_sampled.iloc[val_idx].reset_index(drop=True)
# y_train, y_val = train_data_sampled['likes_capped'].iloc[train_idx].reset_index(drop=True), train_data_sampled['likes_capped'].iloc[val_idx].reset_index(drop=True)

# # Ensure stratification within the group-based split
# # print(f"Training set class distribution: {np.bincount(y_train)}")
# # print(f"Validation set class distribution: {np.bincount(y_val)}")

In [33]:
# Load the data
df = train_data_sampled.copy()

# Preprocess text and combine data
print("Preprocessing train text...")
df['processed_content'] = df['content'].apply(preprocess_text)
df['hashtags'] = df['content'].apply(extract_hashtags)
df['emojis'] = df['content'].apply(extract_emoji)
df['combined_text'] = df.apply(lambda row: f"tweet related to: {row['username']} is {row['processed_content']} {row['hashtags']} {row['emojis']}", axis=1)

Preprocessing train text...


In [ ]:
from torch.utils.data import DataLoader
batch_size = 64

# Create DataLoader for the text data
dataloader = DataLoader(df['combined_text'].tolist(), batch_size=batch_size, shuffle=False)

print("Generating embeddings...")
embeddings = []

# Loop through the batches and generate embeddings
for batch in tqdm(dataloader, desc="Generating text embeddings"):
    batch_embeddings = get_roberta_embedding_batch(batch)
    embeddings.extend(batch_embeddings)

# Add the embeddings to the dataframe
df['combined_embedding'] = embeddings

Generating embeddings...


Generating text embeddings:  36%|███▌      | 17/47 [00:02<00:04,  7.13it/s]

In [ ]:
# Generate other embeddings
print("Generating other embeddings...")
df['datetime_embedding'] = df['date'].apply(get_datetime_features)

# company_encoder = OneHotEncoder(sparse=False)
# company_encoder.fit(df['inferred company'].values.reshape(-1, 1))
# df['company_embedding'] = df['inferred company'].apply(lambda x: get_categorical_embedding(x, company_encoder))

In [ ]:
import re
import numpy as np
import pandas as pd
from collections import Counter
from transformers import pipeline, AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
import torch
from tqdm import tqdm  # For progress tracking

# Load the training data
# df = pd.read_csv("train_tweets.csv")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check for GPU

# 1. Tokenize usernames and perform word frequency analysis
def tokenize_username(username):
    username = username.replace('_', ' ')
    username_parts = re.findall(r'[A-Z][a-z]*|[a-z]+|\d+', username)
    return [word.lower() for word in username_parts]

df['tokenized_username'] = df['username'].apply(tokenize_username)

# 5. Username Embedding Generation using Pre-trained Model
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)  # Move model to GPU

def get_username_embedding_batch(usernames):
    tokenized = tokenizer(usernames, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**tokenized)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Batch size for processing
batch_size = 64

# Tokenize usernames and store them in a list
tokenized_usernames = [' '.join(tokenize_username(username)) for username in df['username'].tolist()]

# Create DataLoader for the usernames
dataloader = DataLoader(tokenized_usernames, batch_size=batch_size, shuffle=False)

# Generate embeddings in batches
print("Generating username embeddings...")
embeddings = []

for batch in tqdm(dataloader, desc="Embedding Generation"):
    batch_embeddings = get_username_embedding_batch(batch)
    embeddings.extend(batch_embeddings)

# Add embeddings back to the DataFrame
df['username_embedding'] = embeddings
# 6. Dimensionality Reduction with PCA
pca = PCA(n_components=64)
df['reduced_username_embedding'] = list(pca.fit_transform(np.stack(df['username_embedding'])))

# 7. Feature List
print("Training features generated successfully!")
print("Feature columns for use in neural network:")
print(df.columns)

In [ ]:
# Combine all embeddings
print("Combining embeddings...")
df['rich_embedding'] = df.apply(lambda row: np.concatenate([
    row['combined_embedding'],
    row['datetime_embedding'],
    row['reduced_username_embedding']
    # row['company_embedding']
    # row['user_graph_embedding'],
    # row['company_graph_embedding']
]), axis=1)

print(f"Shape of rich embedding: {df['rich_embedding'].iloc[0].shape}")

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
# # Load the data
# df = X_val

# # Preprocess text and combine data
# print("Preprocessing val text...")
# df['processed_content'] = df['content'].apply(preprocess_text)
# df['hashtags'] = df['content'].apply(extract_hashtags)
# df['emojis'] = df['content'].apply(extract_emoji)
# df['combined_text'] = df.apply(lambda row: f"{row['processed_content']} {row['username']} {row['inferred company']} {row['hashtags']} {row['emojis']}", axis=1)

In [ ]:
# # Generate embeddings with progress bar
# print("Generating embeddings...")
# embeddings = []
# for text in tqdm(df['combined_text'], desc="Generating text embeddings"):
#     embeddings.append(get_roberta_embedding(text))
# df['combined_embedding'] = embeddings

In [ ]:
# # Generate other embeddings
# print("Generating other embeddings...")
# df['datetime_embedding'] = df['date'].apply(get_datetime_features)

# # company_encoder = OneHotEncoder(sparse=False)
# # company_encoder.fit(df['inferred company'].values.reshape(-1, 1))
# df['company_embedding'] = df['inferred company'].apply(lambda x: get_categorical_embedding(x, company_encoder))

In [ ]:
# print("Creating graph embeddings...")
# df = create_graph_embedding(df)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Prepare data for regression
# X = np.stack(df['rich_embedding'])
y = df['likes_capped'].values

# Log transform the target variable to handle skewness
y = np.log1p(y)  # log1p is used to handle zero values

# Split the data
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
import preprocessor as p
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from textblob import TextBlob

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

# Load English words for username cleaning
english_words = set(nltk.corpus.words.words())

# Preprocess tweets
def preprocess_tweet(tweet):
    # Count mentions and hyperlinks
    mention_count = tweet.count('<mention>')
    hyperlink_count = tweet.count('<hyperlink>')
    
    # Remove mentions and hyperlinks
    tweet = tweet.replace('<mention>', '').replace('<hyperlink>', '')
    
    # Clean the tweet
    tweet = p.clean(tweet)
    # Tokenize
    tokens = word_tokenize(tweet.lower())
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens), mention_count, hyperlink_count

# Clean username
def clean_username(username):
    # Remove special characters and numbers
    clean_name = re.sub(r'[^a-zA-Z]', ' ', username)
    # Split into parts
    parts = clean_name.split()
    # Keep only parts that are likely to be meaningful words
    meaningful_parts = [part.lower() for part in parts if len(part) > 2 and (part.lower() in english_words or not part.isalpha())]
    return ' '.join(meaningful_parts)

# Load your data (assuming you have a CSV with 'content' and 'username' columns)
df = train_data_sampled.copy()

# Preprocess tweets and usernames
df['processed_content'], df['mention_count'], df['hyperlink_count'] = zip(*df['content'].apply(preprocess_tweet))
df['processed_username'] = df['username'].apply(clean_username)

# Combine processed content and processed username
df['text'] = df['processed_content'] + ' ' + df['processed_username']
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df['sentiment'] = df['text'].apply(get_sentiment)
# Split the data first to prevent data leakage
train_df, val_df, y_train, y_val = train_test_split(df,y, test_size=0.2, random_state=42)
train_df, val_df = train_df.reset_index(drop=True), val_df.reset_index(drop=True)

# Perform topic modeling on the training set
def perform_topic_modeling(train_texts, val_texts, n_topics=10):
    # LDA
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    train_doc_term_matrix = vectorizer.fit_transform(train_texts)
    val_doc_term_matrix = vectorizer.transform(val_texts)
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    train_lda_output = lda.fit_transform(train_doc_term_matrix)
    val_lda_output = lda.transform(val_doc_term_matrix)
    
    # BERTopic
    topic_model = BERTopic(nr_topics=n_topics)
    train_topics, _ = topic_model.fit_transform(train_texts)
    val_topics, _ = topic_model.transform(val_texts)
    
    return train_lda_output, val_lda_output, train_topics, val_topics

train_lda, val_lda, train_bertopics, val_bertopics = perform_topic_modeling(train_df['content'], val_df['content'])

# Add topic columns to the dataframes
for i in range(train_lda.shape[1]):
    train_df[f'lda_topic_{i}'] = train_lda[:, i]
    val_df[f'lda_topic_{i}'] = val_lda[:, i]

train_df['bertopic'] = train_bertopics
val_df['bertopic'] = val_bertopics
feature_columns = [col for col in train_df.columns if col.startswith('lda_topic_')] + ['mention_count', 'hyperlink_count', 'sentiment', 'bertopic']

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
# Combine all embeddings
print("Combining embeddings...")
train_df['rich_embedding'] = train_df.apply(lambda row: np.concatenate([
    row['combined_embedding'],
    row['datetime_embedding'],
    
    # row['company_embedding']
    # row['user_graph_embedding'],
    # row['company_graph_embedding']
]), axis=1)
train_df['other_embeddings'] = train_df.apply(lambda row: np.concatenate([
    row[cols] for cols in feature_columns
]), axis=1)
train_df['final_embedding'] = train_df.apply(lambda row: np.concatenate([
    row['rich_embedding'],
    row['other_embeddings'],
    
    # row['company_embedding']
    # row['user_graph_embedding'],
    # row['company_graph_embedding']
]), axis=1)
print(f"Shape of final embedding: {train_df['final_embedding'].iloc[0].shape}")

In [ ]:
# Combine all embeddings
print("Combining embeddings...")
val_df['rich_embedding'] = val_df.apply(lambda row: np.concatenate([
    row['combined_embedding'],
    row['datetime_embedding'],
    
    # row['company_embedding']
    # row['user_graph_embedding'],
    # row['company_graph_embedding']
]), axis=1)
val_df['other_embeddings'] = val_df.apply(lambda row: np.concatenate([
    row[cols] for cols in feature_columns
]), axis=1)
val_df['final_embedding'] = val_df.apply(lambda row: np.concatenate([
    row['rich_embedding'],
    row['other_embeddings'],
    
    # row['company_embedding']
    # row['user_graph_embedding'],
    # row['company_graph_embedding']
]), axis=1)
print(f"Shape of final embedding: {val_df['final_embedding'].iloc[0].shape}")val_

In [ ]:
# X_train = np.stack(train_df['rich_embedding'])


In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# Define a neural network for regression
class TweetLikesRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(TweetLikesRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x.squeeze()

# Create dataset and dataloader
class TweetDataset(Dataset):
    def __init__(self, embeddings, likes):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.likes = torch.tensor(likes, dtype=torch.float32)
    
    def __len__(self):
        return len(self.likes)
    
    def __getitem__(self, idx):
        return self.embeddings[idx], self.likes[idx]

# Hyperparameters
input_dim = X_train.shape[1]
hidden_dim = 256
batch_size = 64
num_epochs = 100
learning_rate = 0.001

In [ ]:
# Create model, optimizer, and loss function
model = TweetLikesRegressor(input_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [ ]:
# Create datasets and dataloaders
train_dataset = TweetDataset(X_train_scaled, y_train)
val_dataset = TweetDataset(X_val_scaled, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
# Training loop
print("Training regression model...")
best_val_mse = float('inf')
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_embeddings, batch_likes in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_embeddings, batch_likes = batch_embeddings.to(device), batch_likes.to(device)
        optimizer.zero_grad()
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_likes)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # Validation
    model.eval()
    val_preds = []
    val_true = []
    with torch.no_grad():
        for batch_embeddings, batch_likes in val_loader:
            batch_embeddings, batch_likes = batch_embeddings.to(device), batch_likes.to(device)
            outputs = model(batch_embeddings)
            val_preds.extend(outputs.cpu().numpy())
            val_true.extend(batch_likes.cpu().numpy())
    
    val_mse = mean_squared_error(val_true, val_preds)
    val_mae = mean_absolute_error(val_true, val_preds)
    val_r2 = r2_score(val_true, val_preds)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")
    print(f"Val MSE: {val_mse:.4f}, Val MAE: {val_mae:.4f}, Val R2: {val_r2:.4f}")
    
    # Save the best model
    if val_mse < best_val_mse:
        best_val_mse = val_mse
        torch.save(model.state_dict(), 'best_tweet_likes_regressor.pth')
        print("New best model saved!")

print("Training completed!")

In [ ]:
# Function to get predictions for new data
def predict_likes(new_data):
    model.eval()
    with torch.no_grad():
        new_data_scaled = scaler.transform(new_data)
        embeddings = torch.tensor(new_data_scaled, dtype=torch.float32).to(device)
        outputs = model(embeddings)
        # Convert log predictions back to original scale
        return np.expm1(outputs.cpu().numpy())

# Example usage:
# new_likes_predictions = predict_likes(new_embeddings)
model.load_state_dict(torch.load('best_tweet_likes_regressor.pth'))
# Evaluate on test set
model.eval()
test_preds = []
test_true = y_val  # Using validation set as test set for this example
with torch.no_grad():
    for batch_embeddings, batch_likes in val_loader:
        batch_embeddings = batch_embeddings.to(device)
        outputs = model(batch_embeddings)
        test_preds.extend(outputs.cpu().numpy())

# Convert predictions back to original scale
test_preds = np.expm1(test_preds)
test_true = np.expm1(test_true)

# Calculate metrics on original scale
test_mse = mean_squared_error(test_true, test_preds)
test_mae = mean_absolute_error(test_true, test_preds)
test_r2 = r2_score(test_true, test_preds)

print("\nTest Set Evaluation (Original Scale):")
print(f"MSE: {test_mse:.2f}")
print(f"MAE: {test_mae:.2f}")
print(f"R2: {test_r2:.4f}")

# Calculate custom metrics for skewed data
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def median_absolute_percentage_error(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(test_true, test_preds)
mdape = median_absolute_percentage_error(test_true, test_preds)

print(f"MAPE: {mape:.2f}%")
print(f"MdAPE: {mdape:.2f}%")

In [ ]:
temp_df = pd.DataFrame(test_preds)
temp_df.describe()

In [ ]:
temp_df = pd.DataFrame(test_true)
temp_df.describe()

In [ ]:
# Calculate Pearson correlation coefficient
correlation = np.corrcoef(test_preds, test_true)[0,1]

print(f"Pearson correlation: {correlation}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(test_preds,test_true)

In [ ]:
# Prepare data for classification
X = np.stack(df['rich_embedding'])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['likes_binned'])  # Assuming 'y' column exists with class labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define a simple neural network for classification
class TweetClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TweetClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Create dataset and dataloader
class TweetDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Hyperparameters
input_dim = X_train.shape[1]
hidden_dim = 256
num_classes = len(np.unique(y))
batch_size = 64
num_epochs = 100
learning_rate = 0.001

In [ ]:
# Create model, optimizer, and loss function
model = TweetClassifier(input_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Create datasets and dataloaders
train_dataset = TweetDataset(X_train, y_train)
val_dataset = TweetDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
# Training loop
print("Training classification model...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_embeddings, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # Validation
    model.eval()
    val_preds = []
    val_true = []
    with torch.no_grad():
        for batch_embeddings, batch_labels in val_loader:
            batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
            outputs = model(batch_embeddings)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_true.extend(batch_labels.cpu().numpy())
    
    val_accuracy = accuracy_score(val_true, val_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(val_true, val_preds, average='weighted')
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

print("Training completed!")

# Save the model
torch.save(model.state_dict(), 'tweet_classifier.pth')
print("Model saved successfully!")

# Function to get predictions for new data
def predict(new_data):
    model.eval()
    with torch.no_grad():
        embeddings = torch.tensor(new_data, dtype=torch.float32).to(device)
        outputs = model(embeddings)
        _, preds = torch.max(outputs, 1)
    return label_encoder.inverse_transform(preds.cpu().numpy())

# Example usage:
# new_predictions = predict(new_embeddings)